In [2]:
import gzip

import pysam
import numpy as np
import pandas as pd
from intervaltree import Interval, IntervalTree

In [ ]:
ref_introns = '/datasets2/lariat_mapping/testing/output/test_ref/introns.tsv.gz'
output_base = '/datasets2/lariat_mapping/testing/100k_truncs/C22-1/'

In [3]:
introns = pd.read_csv(ref_introns, sep='\t')
introns

,chrom,strand,start,end,gene_id
0,chr1,+,12227,12612,ENSG00000290825.1
1,chr1,+,12721,13220,ENSG00000290825.1
2,chr1,+,30039,30563,ENSG00000243485.5
3,chr1,+,30667,30975,ENSG00000243485.5
4,chr1,+,57653,58699,ENSG00000290826.1
...,...,...,...,...,...
389596,chrY,-,25378815,25378819,ENSG00000228786.5
389597,chrY,-,25378878,25386916,ENSG00000228786.5
389598,chrY,-,25387085,25391260,ENSG00000228786.5
389599,chrY,-,25387085,25394625,ENSG00000228786.5


In [5]:
introns['interval'] = introns[['start', 'end', ]].apply(Interval)
introns

0

In [4]:
linear_reads = []
unmapped_rids = set()
for align in pysam.AlignmentFile(f'{output_base}/mapped_reads.bam', 'rb'):
	if align.is_unmapped:
		unmapped_rids.add(align.query_name)
		continue

	linear_reads.append([
					align.query_name, 
					align.reference_name, 
					# align.reference_start,
					# align.reference_end, 
					align.get_blocks(),
					align.cigartuples,
					# align.get_tags(),
					align.is_read1,
					])

linear_reads = pd.DataFrame(linear_reads, columns=LINEAR_COLUMNS)
linear_reads.cigar = linear_reads.cigar.transform(lambda cigar: tuple((CIGARTUPLE_CODES[op],length) for op, length in cigar))
# linear_reads.tags = linear_reads.tags.transform(lambda tags: {tag: val for tag, val in tags})
if single_end is False:
	linear_reads.read = linear_reads.read.map({True: '1', False: '2'})
	linear_reads['read_id'] = linear_reads.read_id + '/' + linear_reads.read
assert linear_reads.read_id.is_unique
linear_reads = linear_reads.sort_values(['read_id'])
linear_reads

NameError: name 'output_base' is not defined